# VESC Motor Controller - Basic Usage

This notebook demonstrates basic usage of the VESC student API for reading telemetry data and controlling the motor.

## Setup

First, let's import the student API and create a connection to the VESC controller.

In [ ]:
# Setup Python path to find student_api from parent directory
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from student_api import VESCStudentAPI
import time

# Create API instance
vesc_api = VESCStudentAPI()

# Start the VESC system
if vesc_api.start():
    print("VESC system started successfully!")
    
    # Get controller for VESC ID 74 (as specified in README)
    vesc = vesc_api.get_controller(74)
    
    if vesc:
        print("Connected to VESC controller!")
    else:
        print("Failed to get VESC controller")
else:
    print("Failed to start VESC system")

## Reading Basic Telemetry

Let's read some basic motor telemetry data:

In [ ]:
# Read current motor status
rpm = vesc.get_rpm()
current = vesc.get_motor_current()
duty_cycle = vesc.get_duty_cycle()
voltage = vesc.get_input_voltage()

print(f"Motor RPM: {rpm}")
print(f"Motor Current: {current} A")
print(f"Duty Cycle: {duty_cycle}")
print(f"Input Voltage: {voltage} V")

## Reading Temperature Data

Monitor motor and controller temperatures:

In [ ]:
# Read temperature data
fet_temp = vesc.get_fet_temperature()
motor_temp = vesc.get_motor_temperature()

print(f"Controller (FET) Temperature: {fet_temp}°C")
print(f"Motor Temperature: {motor_temp}°C")

# Check if temperatures are safe
if fet_temp and fet_temp > 80:
    print("⚠️ WARNING: Controller temperature is high!")
if motor_temp and motor_temp > 100:
    print("⚠️ WARNING: Motor temperature is high!")

## Power and Energy Monitoring

Monitor power consumption and energy usage:

In [ ]:
# Read power and energy data
input_current = vesc.get_input_current()
amp_hours = vesc.get_amp_hours_consumed()
watt_hours = vesc.get_watt_hours_consumed()

print(f"Input Current: {input_current} A")
print(f"Energy Consumed: {amp_hours} Ah, {watt_hours} Wh")

# Calculate approximate power
if voltage and input_current:
    power = voltage * input_current
    print(f"Approximate Power: {power:.2f} W")

## Live Data Monitoring

Get all telemetry data at once and display it continuously:

In [ ]:
# Monitor live data for 10 seconds
import time
from IPython.display import clear_output

print("Monitoring live data for 10 seconds...")
start_time = time.time()

while time.time() - start_time < 10:
    clear_output(wait=True)
    
    # Get all telemetry at once
    data = vesc.get_all_telemetry()
    
    print("=== VESC Live Data ===")
    print(f"RPM: {data['motor']['rpm']}")
    print(f"Current: {data['motor']['current']} A")
    print(f"Duty Cycle: {data['motor']['duty_cycle']}")
    print(f"Voltage: {data['power']['input_voltage']} V")
    print(f"FET Temp: {data['temperatures']['fet']}°C")
    print(f"Motor Temp: {data['temperatures']['motor']}°C")
    
    time.sleep(0.5)

print("Live monitoring complete!")

## Safe Motor Control

**⚠️ WARNING: Only run these commands when it's safe to do so!**

The following cells demonstrate motor control. Make sure the motor is properly secured and safe to operate.

In [ ]:
# SAFETY CHECK - Uncomment the next line to enable motor control
# MOTOR_CONTROL_ENABLED = True
MOTOR_CONTROL_ENABLED = False

if not MOTOR_CONTROL_ENABLED:
    print("Motor control is DISABLED for safety.")
    print("Set MOTOR_CONTROL_ENABLED = True to enable motor commands.")

In [ ]:
if MOTOR_CONTROL_ENABLED:
    # Set a small duty cycle (10%)
    print("Setting duty cycle to 10%...")
    success = vesc.set_duty_cycle(0.1)
    print(f"Command sent: {success}")
    
    # Wait and monitor
    time.sleep(2)
    rpm = vesc.get_rpm()
    print(f"Current RPM: {rpm}")
    
    # Stop the motor
    print("Stopping motor...")
    vesc.set_duty_cycle(0)
    print("Motor stopped.")
else:
    print("Motor control commands skipped (safety mode).")

## Cleanup

Always ensure the motor is stopped when finished:

In [ ]:
# Ensure motor is stopped
vesc.set_duty_cycle(0)
print("Session complete. Motor stopped.")